In [1]:
import ee
import geemap
import ipywidgets as widgets
from datetime import datetime, timedelta

In [5]:
ee.Initialize()

Map = geemap.Map(center=[48.6, 37.46], zoom=8)
Map.add_basemap('OpenStreetMap')

# Styl KML
kml_file = r"C:\Users\slesi\Desktop\Russian_fortifications.kml"
style = {
    'color': 'red',
    'fillColor': 'red',
    'opacity': 0.8,
    'fillOpacity': 0.5
}
Map.add_kml(kml_file, layer_name='Fortyfikacje KML', style=style)

# Parametry
buffer_m = 5000
sar_vis_params = {
    "min": -20,
    "max": 0,
    "bands": ["VV", "VH", "VV"]
}
years = range(2020, 2026)
months = range(1, 13)

# Funkcja generująca kolekcję czasową Sentinel-1
def create_time_series(point):
    roi = point.buffer(buffer_m).bounds()
    images = []
    labels = []

    for year in years:
        for month in months:
            start = ee.Date.fromYMD(year, month, 1)
            end = start.advance(1, 'month')

            collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                .filterBounds(roi) \
                .filterDate(start, end) \
                .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')) \
                .filter(ee.Filter.eq('resolution_meters', 10)) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                .select(['VV', 'VH'])

            image = collection.mean().clip(roi).set('label', f"{year}-{month:02}")
            images.append(image)
            labels.append(f"{year}-{month:02}")
    
    return ee.ImageCollection.fromImages(images), labels

# Przycisk i callback
load_button = widgets.Button(description="Załaduj obrazy Sentinel-1")

def on_button_click(b):
    if Map.user_roi is None:
        print("⚠️ Najpierw narysuj punkt (Toolbox → Draw → Point)")
        return
    
    geom = Map.user_roi
    point = geom.centroid()

    # Wyczyść wszystkie warstwy oprócz KML + basemap
    Map.layers = Map.layers[:2]

    ic, names = create_time_series(point)

    Map.ts_inspector(
        left_ts=ic,
        right_ts=ic,
        left_names=names,
        right_names=names,
        left_vis=sar_vis_params,
        right_vis=sar_vis_params,
    )

    coords = point.coordinates().getInfo()
    Map.setCenter(coords[0], coords[1], 10)

load_button.on_click(on_button_click)

# Wyświetlenie
display(load_button)
Map


Button(description='Załaduj obrazy Sentinel-1', style=ButtonStyle())

Map(center=[48.6, 37.46], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…